In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class ResSect(nn.Module):
    def __init__(self, n_filter, n_residual_blocks, beginning_stride):
        super(ResSect, self).__init__()
        self.blocks = nn.ModuleList()
        for i in range(n_residual_blocks):
            in_channels = n_filter if i == 0 else n_filter * 2
            out_channels = n_filter * 2
            stride = beginning_stride if i == 0 else 1

            self.blocks.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False))
            self.blocks.append(nn.BatchNorm2d(out_channels))
            self.blocks.append(nn.ReLU(inplace=True))
            self.blocks.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False))
            self.blocks.append(nn.BatchNorm2d(out_channels))

    def forward(self, x):
        identity = x
        for block in self.blocks:
            x = block(x)
        if identity.shape[1] != x.shape[1]:
            identity = nn.Conv2d(identity.shape[1], x.shape[1], kernel_size=1, stride=1, bias=False)(identity)
        if identity.shape[2:] != x.shape[2:]
            identity = F.interpolate(identity, size=x.shape[2:], mode='nearest')
        x += identity
        x = nn.ReLU(inplace=True)(x)
        return x

In [4]:
class ResModel(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()

        self.sect1 = ResSect(32, 3, 1)
        self.sect2 = ResSect(64, 3, 2)
        self.sect3 = ResSect(128, 3, 2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(256, 10)

        if pretrained:
            self.load_trained_model()

    def load_trained_model(self):
        state_dict = torch.load("Mymodel.pth")
        self.load_state_dict(state_dict)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.sect1(x)
        x = self.sect2(x)
        x = self.sect3(x)

        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResModel(pretrained=False).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
# Training the model
model.train()
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Given groups=1, weight of size [64, 32, 1, 1], expected input[128, 64, 32, 32] to have 32 channels, but got 64 channels instead

In [ ]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
model.save("Mymodel.pth")